# CovDrugSim Usage Demonstration

This is a demonstration of using `CovDrugSim` to automate the quantum mechanical calculations and molecular dynamics simulations.

The directory and file paths in this notebook are set up for the notebook to be run from sphractal/docs/. If this is not the case, the path needs to be changed accordingly for the cells to be executed properly. 

In [11]:
%load_ext autoreload
%autoreload 2

## Quantum Mechanical Calculations
`CovDrugSim` can be used to automatically:
* group individual `xyz` coordinate files into individual directories, 
* generate job scripts for the `Gaussian` software commonly used for quantum mechanical calculations of small molecules,
* generate job scripts for a given high-performance computing (HPC) system, such as Gadi at National Computational Infrastructure (NCI) in Australia, 
* submit the generated HPC job scripts to the HPC scheduler.
* tabulate quantities of interest from Gaussian output files to an Excel document with different sheets.

### Generation of Gaussian job scripts and HPC submission scripts
Say we have a directory containing xyz files that we want to conduct quantum mechanical calculations on. The directory could have just 1, or 100,000,000... files, the bottleneck is simply how many your system can hold. But importantly, it should only contain xyz files, that you want to run Gaussian jobs on.

We will create the directory with example xyzs in it right now:

In [13]:
from covdrugsim import genExampleXYZs

inpDirPath = './exampleXYZs'
genExampleXYZs(inpDirPath)

In [16]:
!ls -R --color exampleXYZs

exampleXYZs:
example1.xyz  example2.xyz  example3.xyz


We can then simply generate the scripts for them by providing some arguments to the function `genAllScripts()`, we turned on the `verbose` argument to show what's happening under the hood:

In [17]:
from covdrugsim import genAllScripts

genAllScripts(inpDirPath, verbose=True)


Generating all job scripts for molecules under directories under ./exampleXYZs...

Grouping molecules in ./exampleXYZs into individual directory...
  Making directories for molecules in ./exampleXYZs...
    Making directory for example1.xyz...
      Made directory for example1.xyz!
    Making directory for example2.xyz...
      Made directory for example2.xyz!
    Making directory for example3.xyz...
      Made directory for example3.xyz!
  DONE -- Made all directories!

  Moving molecules in ./exampleXYZs into individual directory...
    Processing example1.xyz...
      Moved example1.xyz to example1!
    Processing example2.xyz...
      Moved example2.xyz to example2!
    Processing example3.xyz...
      Moved example3.xyz to example3!
  DONE -- Moved all files!

DONE -- Grouped all molecules!

  Processing example1...
    Generated Gaussian input file for example1!
    Generated HPC job script for example1!
  Processing example2...
    Generated Gaussian input file for example2!
  

We can see that the scripts are now generated by listing `inpDirPath` recursively:

In [20]:
!ls -R --color exampleXYZs

exampleXYZs:
example1  example2  example3

exampleXYZs/example1:
example1.inp  example1.sh  example1.xyz

exampleXYZs/example2:
example2.inp  example2.sh  example2.xyz

exampleXYZs/example3:
example3.inp  example3.sh  example3.xyz


Now `genAllScripts()` take in a lot of parameters, most of which has default values, so if you don't specify them the default values will be used. Note that if you have provided the `keywordLine` argument, a few of the others will be overridden (such as `solvent`, `solventModels`), because they've already been specified in your `keywordLine` argument.

In [6]:
help(genAllScripts)

Help on function genAllScripts in module covdrugsim.qmcalc.genScripts:

genAllScripts(inpDirPath, keywordLine=None, method='m062x', basisSet='6-311+g(d,p)', solvent='water', solventModel='cpcm', mem=4000, ncpus=8, calcType='GOVF', charge=0, spin=1, scheduler='pbs', cluster='gadi', walltime='24:00:00', vmem=8000, jobfs=9000, project='p39', software='g16', version='c01', verbose=False)
    Generate Gaussian input job files and submission files for molecules under all directories under a specified directory ('inpDirPath').
    
    Parameters
    ----------
    inpDirPath : str
        Directory path to the input directories.
    keywordLine : Union[str,None]
        The line of keywords specification for Gaussian job, the other input arguments will be used to compose the line if it is not provided.
    method : str
        Keyword for DFT method specification in Gaussian.
    basisSet : str
        Keyword for basis set specification in Gaussian.
    solvent : str
        Keyword for sol

Below is an example of specifying the keywords through the `keywordLine` argument:

In [7]:
keywordLine = '# m062x/6-311+g(d,p) opt=calcfc freq scrf=(cpcm,solvent=water) int(grid=ultrafine)'
genAllScripts(inpDirPath, keywordLine, verbose=True)


Generating all job scripts for molecules under directories under ./exampleXYZs...
  Processing example1...
    Generated Gaussian input file for example1!
    Generated HPC job script for example1!
  Processing example2...
    Generated Gaussian input file for example2!
    Generated HPC job script for example2!
  Processing example3...
    Generated Gaussian input file for example3!
    Generated HPC job script for example3!
DONE -- Generated all scripts!



## Molecular Dynamics Simulations
